In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=106ba89fe7cbc8d729f007b0556d8cb6acd668f3a0f99827cc3030e9ed9e78bd
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,avg,sum,max

spark=SparkSession.builder.appName("FoodDelivery").getOrCreate()

food_delivert_df=spark.read.csv("/content/food_data.csv", header=True,inferSchema=True)

In [3]:
# 1. Calculate Total Revenue per Restaurant
total_revenue_per_restaurant = food_delivert_df.withColumn("total_revenue", col("price") * col("quantity")) \
    .groupBy("restaurant_name").agg(sum("total_revenue").alias("total_revenue"))
print("total revenue per restaurant: ")
total_revenue_per_restaurant.show()

total revenue per restaurant: 
+---------------+------------------+
|restaurant_name|     total_revenue|
+---------------+------------------+
|         Subway|              13.0|
|      Pizza Hut|             12.99|
|    Burger King|              6.99|
|            KFC|             35.96|
|       Domino's|             23.98|
|     McDonald's|20.950000000000003|
|      Starbucks|              13.5|
+---------------+------------------+



In [4]:
# 2.  Find the Fastest Delivery
fastest_delivery=food_delivert_df.orderBy(col("delivery_time_mins").asc()).limit(1)
print("fastest delivery: ")
fastest_delivery.show()

fastest delivery: 
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|order_id|customer_id|restaurant_name|food_item|quantity|price|delivery_time_mins|   order_d|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|       6|        205|      Starbucks|   Coffee|       1|  4.5|                15|2023-06-18|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+



In [5]:
# 3.Calculate Average Delivery Time per Restaurant
avg_delivery_time=food_delivert_df.groupBy("restaurant_name").agg(avg("delivery_time_mins").alias("avg_delivery_time"))
print("average delivery time per restaurant: ")
avg_delivery_time.show()

average delivery time per restaurant: 
+---------------+-----------------+
|restaurant_name|avg_delivery_time|
+---------------+-----------------+
|         Subway|             20.0|
|      Pizza Hut|             45.0|
|    Burger King|             30.0|
|            KFC|             25.0|
|       Domino's|             40.0|
|     McDonald's|             22.5|
|      Starbucks|             17.5|
+---------------+-----------------+



In [6]:
# 4. Filter Orders for a Specific Customer
order_for_specific_customers=food_delivert_df.filter(col("customer_id")==201)
print("order for specific customers: ")
order_for_specific_customers.show()

order for specific customers: 
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|order_id|customer_id|restaurant_name|food_item|quantity|price|delivery_time_mins|   order_d|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+
|       1|        201|     McDonald's|   Burger|       2| 5.99|                30|2023-06-15|
|       4|        201|         Subway| Sandwich|       2|  6.5|                20|2023-06-17|
+--------+-----------+---------------+---------+--------+-----+------------------+----------+



In [7]:
# 5. Find Orders Where Total Amount Spent is Greater Than $20
orders_on_amount_spent=food_delivert_df.withColumn("total_amount", col("price") * col("quantity")) \
    .filter(col("total_amount") > 20)
print("orders where total amount spent is greater than $20: ")
orders_on_amount_spent.show()

orders where total amount spent is greater than $20: 
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+------------+
|order_id|customer_id|restaurant_name|    food_item|quantity|price|delivery_time_mins|   order_d|total_amount|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+------------+
|       3|        203|            KFC|Fried Chicken|       3| 8.99|                25|2023-06-17|       26.97|
|       5|        204|       Domino's|        Pizza|       2|11.99|                40|2023-06-18|       23.98|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+------------+



In [8]:
# 6. Calculate the Total Quantity of Each Food Item Sold
total_quantity_of_each_food_item=food_delivert_df.groupBy("food_item").agg(sum("quantity").alias("total_quantity"))
print("total quantity of each food item sold: ")
total_quantity_of_each_food_item.show()

total quantity of each food item sold: 
+-------------+--------------+
|    food_item|total_quantity|
+-------------+--------------+
|        Fries|             3|
|       Burger|             3|
|     Sandwich|             2|
|        Pizza|             3|
|       Coffee|             3|
|Fried Chicken|             4|
+-------------+--------------+



In [9]:
# 7. Find the Top 3 Most Popular Restaurants by Number of Orders
top_3_restaurants=food_delivert_df.groupBy("restaurant_name").count().orderBy(col("count").desc()).limit(3)
print("top 3 most popular restaurants by number of orders: ")
top_3_restaurants.show()

top 3 most popular restaurants by number of orders: 
+---------------+-----+
|restaurant_name|count|
+---------------+-----+
|            KFC|    2|
|      Starbucks|    2|
|     McDonald's|    2|
+---------------+-----+



In [10]:
# 8. Calculate Total Revenue per Day
total_revenue_per_day=food_delivert_df.withColumn("total_revenue", col("price") * col("quantity")) \
    .groupBy("order_d").agg(sum("total_revenue").alias("total_revenue"))
print("total revenue per day: ")
total_revenue_per_day.show()

total revenue per day: 
+----------+-------------+
|   order_d|total_revenue|
+----------+-------------+
|2023-06-18|        28.48|
|2023-06-16|        12.99|
|2023-06-15|        11.98|
|2023-06-17|        39.97|
|2023-06-20|        15.99|
|2023-06-19|        17.96|
+----------+-------------+



In [11]:
# 9.  Find the Longest Delivery Time for Each Restaurant
longest_delivery_time=food_delivert_df.groupBy("restaurant_name").agg(max("delivery_time_mins"))
print("longest delivery time for each restaurant: ")
longest_delivery_time.show()

longest delivery time for each restaurant: 
+---------------+-----------------------+
|restaurant_name|max(delivery_time_mins)|
+---------------+-----------------------+
|         Subway|                     20|
|      Pizza Hut|                     45|
|    Burger King|                     30|
|            KFC|                     25|
|       Domino's|                     40|
|     McDonald's|                     30|
|      Starbucks|                     20|
+---------------+-----------------------+



In [12]:
# 10.  Create a New Column for Total Order Value
total_order_value=food_delivert_df.withColumn("total_order_value", col("price") * col("quantity"))
print("new column for total order value: ")
total_order_value.show()

new column for total order value: 
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+-----------------+
|order_id|customer_id|restaurant_name|    food_item|quantity|price|delivery_time_mins|   order_d|total_order_value|
+--------+-----------+---------------+-------------+--------+-----+------------------+----------+-----------------+
|       1|        201|     McDonald's|       Burger|       2| 5.99|                30|2023-06-15|            11.98|
|       2|        202|      Pizza Hut|        Pizza|       1|12.99|                45|2023-06-16|            12.99|
|       3|        203|            KFC|Fried Chicken|       3| 8.99|                25|2023-06-17|            26.97|
|       4|        201|         Subway|     Sandwich|       2|  6.5|                20|2023-06-17|             13.0|
|       5|        204|       Domino's|        Pizza|       2|11.99|                40|2023-06-18|            23.98|
|       6|        205|      Starbucks